# Final Project - US Income Simulation

##### Sean Evers, Jacob Blackmore, Jack Kelleher, Haotian Liang

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### Part 1 - Loading in the Datasets

### Federal Income Tax Rate - Lowest Bracket
https://fred.stlouisfed.org/series/IITTRLB

In [ ]:
low_taxes = pd.read_csv("IITTRLB.csv")
low_taxes.index = np.arange(1,47)
low_taxes.columns = ['Year', 'Low Tax Bracket']
low_taxes.head()

### Federal Income Tax Rate - Highest Bracket
https://fred.stlouisfed.org/series/IITTRHB

In [ ]:
high_taxes = pd.read_csv("IITTRHB.csv")
high_taxes.index = np.arange(1,47)
high_taxes.columns = ['Year', 'High Tax Bracket']
high_taxes.head()

### Unemployment Rates
https://fred.stlouisfed.org/series/UNRATE

In [ ]:
unemployment = pd.read_csv("UNRATE-2.csv")
unemployment.index = np.arange(1,47)
unemployment.columns = ['Year', 'Unemployment']
unemployment.head()

### GDP, In Billions of Dollars
https://fred.stlouisfed.org/series/GDP

In [ ]:
gdp = pd.read_csv("GDP-3.csv")
gdp = gdp.iloc[27:,:]
gdp.index = np.arange(1,47)
gdp.head()

### 10 Year Treasury Rates
https://fred.stlouisfed.org/series/DGS10

In [ ]:
treasury = pd.read_csv("DGS10.csv")
treasury.index = np.arange(1,47)
treasury.head()

### CPI
https://fred.stlouisfed.org/series/CPIAUCSL

In [ ]:
cpi = pd.read_csv("CPIAUCSL.csv")
cpi = cpi.iloc[27:,:]
cpi.index = np.arange(1,47)
cpi.head()

### S&P 500 Returns
https://www.macrotrends.net/2526/sp-500-historical-annual-returns

In [ ]:
sp500 = pd.read_csv("sp-500-historical-annual-returns.csv")
sp500 = sp500.iloc[45:-3,:]
sp500.index = np.arange(1,47)
sp500.head()

### Median Weekly Real Wages
https://fred.stlouisfed.org/series/LES1252881600Q

In [ ]:
wages = pd.read_csv("MEPAINUSA672N.csv")
wages = wages.iloc[:46,:]
wages.index = np.arange(1,47)
wages.head()

### Median Sale Price of Houses
https://fred.stlouisfed.org/series/MSPUS

In [ ]:
houses = pd.read_csv("MSPUS-2.csv")
houses = houses.iloc[10:,:]
houses.index = np.arange(1,47)
houses.head()

### Industrial Production
https://fred.stlouisfed.org/series/INDPRO

In [ ]:
indprod = pd.read_csv("INDPRO-2.csv")
indprod = indprod.iloc[:46,:]
indprod.index = np.arange(1,47)
indprod.head()

### Personal Consumption Expenditures
https://fred.stlouisfed.org/series/PCE

In [ ]:
cpe = pd.read_csv("PCE-2.csv")
cpe = cpe.iloc[:46,:]
cpe.index = np.arange(1,47)
cpe.head()

### Personal Income
https://fred.stlouisfed.org/series/MEPAINUSA672N

In [ ]:
p_income = pd.read_csv("MEPAINUSA672N.csv")
p_income.index = np.arange(1,47)
p_income.head()

### Putting Them All Together

In [ ]:
data = p_income

In [ ]:
data.columns = ["Date", "Personal Income"]
data["High Tax Bracket"] = high_taxes.iloc[:,1]
data["Low Tax Bracket"] = low_taxes.iloc[:,1]
data["Unemployment Rate"] = unemployment.iloc[:,1]
data["GDP"] = gdp.iloc[:,1]
data["10 Yr Treasury Yield"] = treasury.iloc[:,1]
data["CPI"] = cpi.iloc[:,1]
data["SP500"] = sp500.iloc[:,1]
data["Wages"] = wages.iloc[:,1]
data["Houses"] = houses.iloc[:,1]
data["Industrial Production"] = indprod.iloc[:,1]
data["Consumer Expenditures"] = cpe.iloc[:,1]
data["Date"] = unemployment.iloc[:,0]
data.head()

### Part 2 - Multiple Linear/Log Regression

In [ ]:
import statsmodels.api as sm
from scipy.stats import pearsonr

In [ ]:
# defining our dependent variable
y = data["Personal Income"]
# defining our independent variables and dropping columns date and personal income
x = data.drop(columns = ["Date", "Personal Income"])

# adding constant for our intercept
x_with_cnst = sm.add_constant(x)

#running the model
model = sm.OLS(y, x_with_cnst)
results = model.fit()
results.summary()

What we see here is that each variable is insignificant except for real wages, but that our adjusted squared value is 1.00. Let's try our model again without real wages:

In [ ]:
# running the model again but dropping wages because it is pretty much the same as our dependent variable
y = data["Personal Income"]
x = data.drop(columns = ["Date", "Personal Income", "Wages"])
x_with_cnst = sm.add_constant(x)

model = sm.OLS(y, x_with_cnst)
results = model.fit()
results.summary()

Our model is looking better now. Just how much do real wages predict changes in median income? Perhaps the two variables are pulled from the same data.

In [ ]:
# running a linear regression of personal income ~ wages.
# if the two datasets are the same, we should expect an R squred value of 1, and correlation of 1
y = data["Personal Income"]
x = data["Wages"]
print(pearsonr(x,y))
x_with_cnst = sm.add_constant(x)

model = sm.OLS(y, x_with_cnst)
results = model.fit()
results.summary()

Because the correlation and R-squared value of our two csv files: Real Wages and Median Personal Income are both 1, this means that our files are pretty much the exact same. So in our other words, our project has just become: How to predict real wage growth. Going forward, we must not use wages in our regression.

### Part 3 - Determining The Best Model

#### Now that we have all of our relevant variables, lets try to come up with the best model.

In [ ]:
def forward_regression(X, y,
                       threshold_in,
                       verbose=False):
    '''The forward regression function determines the most statistically significant variables in a regression.
    X is a dataframe of our independent variables, including a column of constants (1). y is the response variable as
    a dataframe. threshold_in is the p_value that we want our variables to be below in order to be included in our model.
    Set verbose = True in order to view the step by step of variables added to the model and their respective p-value.
    
    Forward regression starts off by doing a regression against each individual variable. Then, we keep only the variable
    the lowest p-value and discard the rest. Then, we run regressions again with our previous variable and the rest of the
    variables, and keep the next variable with the lowest p-value and discard the rest. We keep adding variables until there
    are no variables left with a p-value that is below our threshold_in value.'''
    
    # initilaizing a list that will contain the variables in our model
    initial_list = []
    included = list(initial_list)
    
    # we are going to do a new linear regression until we break out of the loop
    while True:
        
        # setting changed = False. If do not add any more variables to our model, then we break out of the loop because of this
        changed=False
        
        # the variables that are NOT already in our model
        excluded = list(set(X.columns)-set(included))
        
        # initializing new_pval list with length equal to the variables not included in our model
        new_pval = pd.Series(index=excluded)
        
        # for each variable not in our model
        for column in excluded:
            
            # running a regression of our included variables + each variable not in our model
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[column]]))).fit()
            # updating our new_pval list. This will only contain the p_values of our excluded variables not already in the model
            new_pval[column] = model.pvalues[column]
            
        # find the lowest pval from our excluded list
        best_pval = new_pval.min()
        
        # if that lowest pval is less than our threshold:
        if best_pval < threshold_in:
            best_feature = new_pval.idxmin()
            # add it to our included list
            included.append(best_feature)
            
            # set changed = True so that we continue the loop
            changed=True
            
            # if verbose is set to true, outputs which variables are added and the p_value
            if verbose:
                print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))

        # if we go through this process and we do not find any items that 
        # have a p_value less than our threshold, changed will be false, and we break out of the loop
        if not changed:
            break

    return included

In [ ]:
def backward_regression(X, y,
                           threshold_out,
                           verbose=False):
    '''The backward regression function determines the most statistically significant variables in a regression.
    X is a dataframe of our independent variables, including a column of constants (1). y is the response variable as
    a dataframe. (1 - threshold_out) is the p_value that we want our variables to be below in order to be included in our model.
    Set verbose = True in order to view the step by step of variables added to the model and their respective p-value.
    
    Backward regression starts off with all variables in our model. After doing a regression, the highest p-value is
    calculated. If the variable with the highest p-value has a p-value higher than our threshold_out, then we remove it from
    from the model. Do this until all variables have p-values below the threshold_out.'''
    
    # the variables included in our model
    included=list(X.columns)
    
    # run through a loop until we break out
    while True:
        
        # setting changed = False. If do not add any more variables to our model, then we break out of the loop because of this
        changed=False
        # running a regression of our variables
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        
        # if the worst_pval is higher than our threshold:
        if worst_pval > threshold_out:
            
            # set changed = true to continue with the loop
            changed=True
            
            # remove the variable from our model
            worst_feature = pvalues.idxmax()
            included.remove(worst_feature)
            
            #print verbose if verbose = true
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
                
        # break out of the loop if no variables are excluded
        if not changed:
            break
    return included

Source: https://datascience.stackexchange.com/questions/937/does-scikit-learn-have-forward-selection-stepwise-regression-algorithm

In [ ]:
# lets calculate our variables to pass into the functions
y = data["Personal Income"]
x = data.drop(columns = ["Date", "Personal Income", "Wages"])
x_with_cnst = sm.add_constant(x)

In [ ]:
forward_regression(x_with_cnst, y, threshold_in = 0.05)

In [ ]:
backward_regression(x_with_cnst, y, threshold_out = 0.05)

These two models should include variables that are all statistically significant.

### Forward Regression Model

In [ ]:
y = data["Personal Income"]

# we are going to drop all of the variables that are not in our forward model
x = data.drop(columns = ["Date", "High Tax Bracket", "Low Tax Bracket", "Personal Income", "Wages", "CPI", "GDP", "Houses", "Consumer Expenditures"])
x_with_cnst = sm.add_constant(x)

model = sm.OLS(y, x_with_cnst)
results = model.fit()
results.summary()

With the unemployment rate, 10 Yr Treasury Yield, and Industrial Production Index, and the S&P500, we can explain about 96% of the variation in real wages/personal income.

forward_model = 23310 - 412.4072(unemployment) - 179.1327(10 Yr Treasury) - 13.2388(S&P500) + 119.0190(Industrial Production)

In [ ]:
# lets plot our model against actual income
q = gdp.iloc[:,0]
unemployment_rate = data["Unemployment Rate"]
yields = data["10 Yr Treasury Yield"]
sp_500 = data["SP500"]
industrial_production = data["Industrial Production"]

# using the coefficients in our model to calculate median incomes
t = 23310 - 412.4072*(unemployment_rate) - 179.1327*(yields) - 13.2388*(sp_500) + 119.0190*(industrial_production)

In [ ]:
plt.plot(q,t, label = "Model Implied Personal Income")
plt.plot(q, data["Personal Income"], label = "Actual Personal Income")
plt.xticks(np.arange(0, len(x)+1, 10))
plt.title("Forwards Step-Wise Model")
plt.xlabel("Date")
plt.ylabel("Median Personal Income")
plt.legend()

### Backwards Regression Model

In [ ]:
y = data["Personal Income"]
# dropping all variabels not included in our backward regression model
x = data.drop(columns = ["Date", "Low Tax Bracket", "Personal Income", "Wages", "SP500", "Consumer Expenditures", "10 Yr Treasury Yield"])
x_with_cnst = sm.add_constant(x)

model = sm.OLS(y, x_with_cnst)
results = model.fit()
results.summary()

### Plotting Our Model

backwards_model = 29270 -86.3976(high_tax_bracket) -342.9998(unemployment_rate) + 1.1485(gdp) -75.5190(cpi) -0.0375(houses) + 159.7341(industrial_production)

In [ ]:
x = gdp.iloc[:,0]
high_tax_bracket = data["High Tax Bracket"]
unemployment_rate = data["Unemployment Rate"]
gdp = data["GDP"]
cpi = data["CPI"]
houses = data["Houses"]
industrial_production = data["Industrial Production"]
# using our model equation to calculate median income
y = 29270 -86.3976*(high_tax_bracket) -342.9998*(unemployment_rate) + 1.1485*(gdp) -75.5190*(cpi) -0.0375*(houses) + 159.7341*(industrial_production)

In [ ]:
plt.plot(x,y, label = "Model Implied Personal Income")
plt.plot(x, data["Personal Income"], label = "Actual Personal Income")
plt.xticks(np.arange(0, len(x)+1, 10))
plt.title("Backwards Step-Wise Model")
plt.xlabel("Date")
plt.ylabel("Median Personal Income")
plt.legend()